In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
import classifier.multi_dnn_classifier as dnn
from random import randint
from keras import callbacks
from keras import losses
from visualizer import *
from keras.utils import np_utils
from sklearn.metrics import classification_report
import importlib


In [11]:
timeframe_in_ms = {"15m":15*60*1000,"1h":60*60*1000,"1d":24*60*60*1000}

symbol = "BTCUSDT"
trade_tf = "1h"
granular_tf = "1m"

tp = 0.05
sl = 0.025
md = 7


data = pd.read_csv("../data/{}-{}.csv".format(
    symbol,trade_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

granular_data = pd.read_csv("../nocommit/{}-{}.csv".format(
    symbol,granular_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

fm: FeatureManager = FeatureManager()
fm.build_feature(data=data,lags=5)

fm.prepare_trade_forward_data(
    data=fm.df,
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
    granular_data=granular_data,
    timeframe_in_ms=timeframe_in_ms[trade_tf]
)
# loop_classifier(data=fm.df,cols=fm.cols,target_col="trade_signal",laps=3)

In [ ]:
fm.df["trade_signal"].value_counts()

In [ ]:
classifier = dnn.MultiClassDNNClassifer()
classifier.prepare_data(
    data = fm.df,
    cols = fm.cols,
    target_col="trade_signal",
    y_to_categorical=True, 
    rebalance="under",
    random_state=randint(0,100))
classifier.configure(hu=100,input_dim=len(fm.cols),class_num=3)
classifier.run()